[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ArtIC-TITECH/b3-proj-2024/blob/main/docs/theme_A.ipynb)

In [ ]:
## Pytorch関連ライブラリ
import torch
import torchvision
import torchvision.transforms as transforms

## 概要
- 量子化関数の実装
- 量子化を適用したモデルの評価
- 再学習の実装
    - 再学習用の逆伝播の実装
    - モデルの再学習

## 量子化関数の実装

量子化とは、モデルのパラメータや入力を低ビットで表現する手法です。\
低ビットで表現することで計算コストやモデルサイズを削減することができます。\
実際に、エッジデバイス上でニューラルネットワークを使用する際は量子化適用することがほとんどです。\
今回はFP32の値をIntにする量子化を演習を通して行いたいと思います。\
量子化の中でもUniform Quantizationと呼ばれる量子化の幅を均一にした量子化が頻繁に用いられ、\
入力$r$に対して出力$\hat{r}$は次のように計算されます。

$Q(r) = \rm{Int}(r/S) - Z$

$\hat{r} = S(Q(r) + Z)$

$Z$は$0$がどの値に量子化されるかを決めます。非対称の入力を量子化する際はこちらの値を使用して調整します。
$S$は量子化のスケールファクターを表しており、$\alpha$, $\beta$を量子化範囲の最小値、最大値として

$S = \frac{\beta - \alpha}{2^b - 1}$

で計算されます。ここで、$b$は量子化ビット幅と呼ばれます。\
$S$を用いることで量子化の幅を調整して、実際の値と近い値を選ぶことができます。\
今回の演習では簡単のため$\alpha = -\beta = \max(|\max(r)|, |\min(r)|), Z=0$として量子化を行います。\
(余裕がある方は$\alpha, \beta$の範囲を変えて実験してみてください。)\
では、実際にpytorchのテンソルを量子化してみましょう。量子化幅を調整できるよう$r$と$b$を引数として実装してみてください。

pytorchのテンソルの最大値、最小値は次のように得ることができます。

```python
x = torch.rand(1000) # initialize pytorch tensor
x_max, x_min  = x.max(), x.min()
```

数式上の$\rm{Int(x)}$関数はpytorch上では`round`という関数を使用します。
```python
# x.clamp 値域を[-2**{b-1}-1, 2**{b-1}]に制限 (量子化の範囲)
# x.round Intに丸め込みをする関数
x_int = x.round(x.clamp(x, -2**{b-1}-1, 2**{b-1}))
```

In [ ]:
# r: real value (fp32), b: bit-width
def quantization(r: torch.Tensor, b=4):
    # Implement S
    # Implement hat_r
    return 

実装が終わったら入出力のplotを行ってみましょう。
下のセルを実行することで横軸に量子化前の値、縦軸に量子化後の値をplotすることができます。

In [ ]:
import matplotlib.pyplot as plt
torch.manual_seed(100)
r = torch.randn(1000).sort()[0]
hat_r = quantization(r, b=3)

plt.plot(r, hat_r)

### 演習
- ${\alpha, \beta} = {+\rm{std}(x), -\rm{std}(x)}$に変更して入出力がどう変化するか確認してください。
- ビット幅をb=8, 4, 2と変更して入出力がどう変化するか確認してください。

### 量子化を利用したモデルの作成

次に先ほど実装した関数を畳み込み層と線形層に適用してみましょう。\
ここでは、pytorchで実装されているConv2dとLinearクラスを継承した\
QuantConv2dとQuantLinearというクラスを作成します。\
下のセルでinput_quantizerとweight_quantizerを先ほど実装した量子化関数に変更してください。\
本来であればバイアスも量子化する必要がありますが、今回は簡単のため入力と重みのみを量子化します。

In [ ]:
from typing import Union
import torch.nn as nn
from torch.nn.common_types import _size_2_t
import torch.nn.functional as F

class QuantConv2d(nn.Conv2d):
    def __init__(self, in_channels: int, out_channels: int, kernel_size: _size_2_t, stride: _size_2_t = 1, padding: _size_2_t | str = 0, dilation: _size_2_t = 1, groups: int = 1, bias: bool = True, padding_mode: str = 'zeros', device=None, dtype=None) -> None:
        super().__init__(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias, padding_mode, device, dtype)
        self.weight_quantizer = quantization
        # self.weight_quantizer = Quantization
        self.input_quantizer = quantization
        # self.input_quantizer = Quantization
    
    def forward(self, x):
        return self._conv_forward(self.input_quantizer.apply(x), self.weight_quantizer.apply(self.weight), self.bias)

class QuantLinear(nn.Linear):
    def __init__(self, in_features: int, out_features: int, bias: bool = True, device=None, dtype=None) -> None:
        super().__init__(in_features, out_features, bias, device, dtype)
        self.weight_quantizer = quantization
        self.input_quantizer = quantization
    
    def forward(self, x):
        return F.linear(self.input_quantizer.apply(x), self.weight_quantizer.apply(self.weight), self.bias)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # self.conv1 = QuantConv2d(3, 32, 5)
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = QuantConv2d(32, 64, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.bn2 = nn.BatchNorm2d(64)
        # self.fc1 = QuantLinear(64 * 5 * 5, 120)
        self.fc1 = nn.Linear(64 * 5 * 5, 120)
        # self.fc2 = QuantLinear(120, 84)
        self.fc2 = nn.Linear(120, 84)
        # self.fc3 = QuantLinear(84, 10)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class QuantizedNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = QuantConv2d(3, 32, 5)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = QuantConv2d(32, 64, 5)
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1 = QuantLinear(64 * 5 * 5, 120)
        self.fc2 = QuantLinear(120, 84)
        self.fc3 = QuantLinear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net().to('cuda:0')
quantized_net = QuantizedNet().to('cuda:0')

量子化する前の学習済みの重みを読み込みます。

In [ ]:
!curl -L -o model_out.pt https://github.com/ArtIC-TITECH/b3-proj-2023/raw/main/resources/class_03/pretraint.pth

In [ ]:
net.load_state_dict(torch.load('../resources/class_03/pretrain.pth'))
quantized_net.load_state_dict(torch.load('../resources/class_03/pretrain.pth'))

テストデータセットを使用して精度を測ってみましょう。

In [ ]:
## Please update the path to your own directory.
# path=/path/to/your_own  # Uncomment this line
path = '../../work/data/cifar10'

## Define Augmentation
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
     ])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root=path, train=True,
                                        download=True, transform=transform)

n_samples = len(trainset)
trainsize = int(n_samples * 0.8)

trainsubset, validsubset = torch.utils.data.random_split(trainset, [trainsize, n_samples-trainsize])

trainloader = torch.utils.data.DataLoader(trainsubset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

valloader = torch.utils.data.DataLoader(validsubset, batch_size=batch_size, 
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root=path, train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
def accuracy(loader, model):
    total = 0
    correct = 0
    net.eval()
    with torch.no_grad():
        for data in loader:
            images, labels = data[0].to('cuda:0'), data[1].to('cuda:0')
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    net.train()
    return 100.0 * correct / total

def accuracy_batch(outputs, labels):
    total = 0
    correct = 0
    total = labels.size(0)
    _, predicted = torch.max(outputs.data, 1)
    correct = (predicted == labels).sum().item()
    return 100 * correct / total

In [ ]:
print(accuracy(testloader, net))
print(accuracy(testloader, quantized_net))

ビット幅を小さくすると精度が下がることがわかると思います。\
このような場合はfine-tuningと呼ばれる再学習を行うことで精度を回復させることができます。\
しかし、量子化のグラフを見るとわかるように量子化を適用する関数は任意の点で勾配が$0$になってしまうため、\
このままでは学習することができません。そこで、`torch.autograd.Function`クラスを継承したクラスを作成して、\
勾配を近似する必要があります。今回はStraight Through Estimatorと呼ばれる勾配を恒等関数の勾配(つまり全ての入力に対して微分値が$1$になる関数)に近似する方法を用いて量子化関数の微分を定義します。

`quantization`の関数を定義したセルの下に次のようなクラスを作成して量子化の順伝播と逆伝播を定義してください。

```python
class Quantization(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        # Call quantization function
        return quantization(x)
    
    def backward(ctx, x):
        # Implement backward of identity function
        return 

```

Functionクラスを継承したクラスは`apply`という関数を用いて呼び出すのでQuantConv2dとQuantLinearを次のように変更してください。

```python
class QuantConv2d(nn.Conv2d):
    def __init__(self, in_channels: int, out_channels: int, kernel_size: _size_2_t, stride: _size_2_t = 1, padding: _size_2_t | str = 0, dilation: _size_2_t = 1, groups: int = 1, bias: bool = True, padding_mode: str = 'zeros', device=None, dtype=None) -> None:
        ...
        self.weight_quantizer = Quantization
        self.input_quantizer = Quantization
    
    def forward(self, x):
        return self._conv_forward(self.input_quantizer.apply(x), self.weight_quantizer.apply(self.weight), self.bias)
```


実装が終わったらモデルの再学習を行ってみましょう。

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(quantized_net.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-5)

In [ ]:
quantized_net.train()
for epoch in range(5):
    running_loss = 0.
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to('cuda:0'), data[1].to('cuda:0')

        # 1. forward
        outputs = quantized_net(inputs)
        # 2. compute loss
        loss = criterion(outputs, labels)
        
        # 3. reset parameter gradient
        optimizer.zero_grad()

        # 4. backward
        loss.backward()

        # 5. update parameters
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        if i % 100 == 99:
            ## Calculate validation accuracy
            val_acc = accuracy(valloader, quantized_net)

            ## Calculate batch accuracy
            batch_acc = accuracy_batch(outputs=outputs, labels=labels)

            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}, validation accuracy: {val_acc}, batch accuracy: {batch_acc}')

            ## Call writer.add_scaler
            
            running_loss = 0.0

print('Finished Training')

今回の演習ではFunctionクラスを自身で実装して量子化を実装して頂きましたが\
pytorchでは量子化もサポートしているのでもしよければこちらも試してみてください。

https://pytorch.org/docs/stable/quantization.html

## 最終課題
それでは、最後に最終課題の説明をします。\
最終課題では、以下の中の一つの変更をニューラルネットワークに適用することで精度がどのように変化するか調査してください。
資料提出締め切りは11/22(水) 23:59分です。締め切りまでに発表用のスライドをslackにて提出してください。

- データセット変更
- 学習データ拡張
- Optimizer変更
- LR Scheduler追加
- モデル変更
- バッチサイズと学習速度の関係の調査
- 量子化ビット幅変更